# CCKWs event selection

## Import package

In [18]:
import glob
import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt

## Load data

In [19]:
# experiment names
exp_name: list[str] = ["CNTL", "NCRF", "NSC"]

# file name
fname: list[str] = [
    f"/work/b11209013/2024_Research/MPAS/merged_data/{exp}/q1.nc"
    for exp in exp_name
    ]

# load dimension
dims: dict[str, np.ndarray] = dict()

with nc.Dataset(fname[0]) as f:
    for dim in f.dimensions.items():
        dims[dim[0]] = np.array(f.variables[dim[0]][:])

lat_lim: tuple[int, int] = np.where((dims["lat"] >= -5) & (dims["lat"] <= 5))[0]

dims["lat"] = dims["lat"][lat_lim]
dims["time"] = dims["time"][:360]

# load data
data: dict[str, np.ndarray] = dict()

for exp, f in zip(exp_name, fname):
    with nc.Dataset(f) as f:
        data[exp]  = np.array(f.variables["q1"][:360, :, lat_lim, :])
        data[exp] -= np.nanmean(data[exp], axis=(0, 3), keepdims=True)

ltime, llev, llat, llon = data["CNTL"].shape

print("Shape of CNTL data", data["CNTL"].shape)
print("Shape of NCRF data", data["NCRF"].shape)
print("Shape of NSC data", data["NSC"].shape)

Shape of CNTL data (360, 38, 20, 720)
Shape of NCRF data (360, 38, 20, 720)
Shape of NSC data (360, 38, 20, 720)


## Vertical Integrate Q1

In [20]:
def vert_int(
        data: np.ndarray,
        levs: np.ndarray,
) -> np.ndarray:
        data_ave : np.ndarray = (data[:, 1:] + data[:, :-1])/2
        data_vint: np.ndarray = data_ave * np.diff(levs*100)[None, :, None, None]

        return -data_vint.sum(axis=1)*86400/9.81/2.5e6

# vertical integration of Q1
q1_vint: dict[str, np.ndarray] = dict(
        (exp, vert_int(data[exp], dims["lev"]))
        for exp in exp_name
        )

print("Shape of CNTL data after vertical integration", q1_vint["CNTL"].shape)



Shape of CNTL data after vertical integration (360, 20, 720)


## Symmetrize Q1 data

In [21]:
def symm(
        data: np.ndarray,
        lats: np.ndarray,
) -> np.ndarray:
        lat_cos: np.ndarray = np.cos(np.deg2rad(lats))

        data_symm: np.ndarray = np.sum((lat_cos[None, :, None] * data) / np.sum(lat_cos), axis=1)

        return data_symm

# symmetrize the data
q1_vint_symm: dict[str, np.ndarray] = dict(
        (exp, symm(q1_vint[exp], dims["lat"]))
        for exp in exp_name
        )

print("Shape of CNTL data after symmetrization", q1_vint_symm["CNTL"].shape)

Shape of CNTL data after symmetrization (360, 720)


## Bandpass filter on vertical integrate Q1

### 2D FFT on the symmetric Q1

In [22]:
def fft2d(
        data: np.ndarray,
) -> np.ndarray:
        data_fft: np.ndarray = np.fft.fft(data, axis=1)
        data_fft: np.ndarray = np.fft.ifft(data_fft, axis=0)

        return data_fft

# 2D FFT
q1_vint_symm_fft: dict[str, np.ndarray] = dict(
        (exp, fft2d(q1_vint_symm[exp]))
        for exp in exp_name
        )

### Bandpass filter

In [23]:
# setup the wavenumber and frequency
wn: np.ndarray = np.fft.fftfreq(llon, d=1/llon).astype(int)
fr: np.ndarray = np.fft.fftfreq(ltime, d=1/4)

wnm, frm = np.meshgrid(wn, fr)

kel_fr = lambda wn, ed: wn * (86400 / (2*np.pi*6.371e6))*np.sqrt(9.81*ed)

# setting up condition of the bandpass filter
kel_lim = np.where(
    ((wnm >= 1) & (wnm <= 14) &
    (frm >= 1/20) & (frm <= 1/2.5) &
    (frm >= kel_fr(wnm, 8)) & (frm <= kel_fr(wnm, 90))) | 
    ((wnm <= -1) & (wnm >= -14) &
    (frm <= -1/20) & (frm >= -1/2.5) &
    (frm <= kel_fr(wnm, 8)) & (frm >= kel_fr(wnm, 90))),
    1, 0)

# bandpass filter
q1_vint_symm_fft_bp: dict[str, np.ndarray] = dict(
        (exp, q1_vint_symm_fft[exp]*kel_lim)
        for exp in exp_name
        )

### Inverse FFT

In [24]:
def ifft2d(
        data: np.ndarray,
) -> np.ndarray:
        data_ifft: np.ndarray = np.fft.ifft(data, axis=1)
        data_ifft: np.ndarray = np.fft.fft(data_ifft, axis=0)

        return data_ifft

# inverse 2D FFT
q1_vint_symm_bp: dict[str, np.ndarray] = dict(
        (exp, ifft2d(q1_vint_symm_fft_bp[exp]).real)
        for exp in exp_name
        )
print("Shape of CNTL data after inverse FFT", q1_vint_symm_bp["CNTL"].shape)

Shape of CNTL data after inverse FFT (360, 720)


In [27]:
sel_event: list[tuple[int, int]] = []


for key in exp_name:
    sel_event = []

    for i in np.linspace(12, 348, 13, dtype=int):
        if q1_vint_symm_bp[key][i, q1_vint_symm_bp[key][i].argmax()] >= np.percentile(q1_vint_symm_bp[key], 97):
            
            sel_event.append((i, q1_vint_symm_bp[key][i].argmax()))

    np.save(f"Selected_Events/q1_sel_{key}.npy", sel_event)
